# Speech recognition task
In this notebook I will guide you in one of the most common challenge of these days, that is, a speech recognition task. This is a **classification** task, our dataset is composed of 105835 .wav recordings with 35 possible labels and for each recording we have Mel-frequencies. Firstly, we will see how to load data and how to process them. Secondly, we will use differnt algorithms to predict our classes. Particularly, we will deal with:

1. **Decision forest**
2. **CNN 2D**
3. **CNN 1D**

### Data processing

The first thing to do is to import all the necessary libraries:

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

import keras
from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.


Now we can start to load the data and then to precess them. Before starting, let's see in more details our data.
you can find data here: https://surfdrive.surf.nl/files/index.php/s/A91xgk7B5kXNvfJ

- **feat.npy** is an array with Mel-frequency cepstral coefficients extracted from each wav file. The features at index *i* in this array were extracted from the wav file at index *i* of the array in the file path.npy.
- **path.npy** is an array with the order of wav files in the feat.npy array.
- **train.csv** contains two columns: path with the filename of the recording and word with word which was pronounced in the recording. This is the training portion of the data.
- **test.csv** is the testing portion of the data, and it has the same format as the file train.csv except that the column word is absent.

In [3]:
                        # --------------------- LOAD DATA ------------------------ #

features = np.load("feat.npy", allow_pickle = True)
path = np.load("path.npy", allow_pickle = True)
train = pd.read_csv("train.csv", delimiter = ",")
test = pd.read_csv("test.csv", delimiter = ",")

Tu put it simply, in path.npy we have the "wav name" of the recordings and in feat.npy we have its features.

In [ ]:
print("The file with name ",path[0], "has this set of features:")
print("----------------------------------------------------------")
print(features[0])

As you can see the index in path and features must be the same, otherwise we are going to misalign "wav file name - features" and consequently, have a very bad accuracy.

Ok, now stay with me, because this is the most boring part of the project. Unfortunatly the data are not set up to work directly with them and we cannot apply ML or DL algortihms (yet), but this is fine, we are Data Scientists and we know that we have to get "our hands dirty" before having fun ;).

The problem is that we already have the split of train set and test set (see train.csv and test.csv). I know, now you are thinking: "Why in the world is this a problem? we need those!". That is true, but the problem is that we just have the "file name" already splitted and not also their features.

In [ ]:
print(test.head())
print()
print(train.head())

In conclusion, we need to find a way to split our data, which are divided in path.npy and feat.npy, according to the indices in train.csv and test.csv.

To do that, we first link our path.npy and feat.npy with a dictionary, key = path and value = feat. You will understand why in a moment. 

In [6]:
                        # ------------------- DATA PROCESSING ---------------------- #

# create dictionary: key = path and value = feat
dic = {} 
for i in range(len(path)):
    dic[path[i]] = features[i]

After creating a dictionary, we need a fuction that allows us to split all our data (now in the dictionary) into train and test sets according to the csv files. Basically, we go through all the data_frame, which are train.csv and test.csv and thanks to the information into the dictionary we can create a list for each data_frame, where in the *ith* position for both, data_frame and dictionary, we have its *i* set of features. 

In [7]:
# this function take as argument a pandas data frame and a dictionary
# and create a new list according to the ith position in the dataframe

def create_list(data_frame,dic):
    new_list= []
    for i in range(len(data_frame)):
        if data_frame["path"][i] in dic.keys():
            new_list.append(dic[data_frame["path"][i]]) # in the position i we add its features thanks to the dic
    return new_list

#in order to convert a list in a numpy array we need to padd our data
def padding(data):
    zeros_list=[0,0,0,0,0,0,0,0,0,0,0,0,0]
    for example in range(len(data)):
        if data[example].shape[0]!=99:
            to_change=data[example].tolist()
            for adding in range(99-len(to_change)):
                to_change.append(zeros_list)
            data[example]=np.array(to_change)     
    return data
    

Now we can apply all the defined functions and we can also have an overview of the shape so that we have a feeling about what kind of data we are going to deal with.

In [5]:
# split test and train 
training_data = create_list(train,dic)
test_data = create_list(test,dic)

# padding
training_data = padding(training_data)
test_data = padding(test_data)

# convert to array
training_data = np.array(training_data)
test_data = np.array(test_data)

#check shape
training_data.shape,test_data.shape

((94824, 99, 13), (11005, 99, 13))

We did it, we have now a ordered training data and test data, respectively compose of 94824 and 11005 examples with 99 lists of 13 elements each. The new way or linkig all the features that we have now in the variables training_data and test_data is with the train.csv and test.csv

In [ ]:
print("This file name", train.loc[0], "\n\n has this features \n" )
print(training_data[0])

### Decision forest

Finally, we can start applying some model, but first we need to encode our target. We will use the function [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html), because our target is a string and it is much easier to work with integer and so we encode our labels into numbers.

In [6]:
                # ------------- set up data for running DECISION FOREST ----------------#

# encode the target
train_numpy = train.values
labels = train_numpy[:,1]
encoder = LabelEncoder()
encoder.fit(labels)
labels = encoder.transform(labels)

We create now a validation data for testing the decision forest thanks to the function [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [7]:
## SPLITTING
X_train, X_test, y_train, y_test = train_test_split(training_data, labels, test_size=0.2, random_state=42)

In [8]:
# check shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(75859, 99, 13)
(75859,)
(18965, 99, 13)
(18965,)


#### Features engineering

In order to improve the accuracy of our decision forest, it is wise to do first some features engineering. We define two functions. The first creates simply the mean of our signal, while the second does something a little bit more complicated, that is, it applys different statistics indices to our signal

In [9]:
                    # ---------- features engineering for decision forest  -------#

def features_mean(signal):
    return np.mean(signal,axis=2)

def features(signal, functions):
    summaries=[]
    for fn in functions:
        summaries.append(fn(signal,axis=2))
    return np.concatenate(summaries,axis=1)

summaries = [np.mean, np.min, np.max, np.std]

X_train_summaries = features(X_train, summaries)
X_test_summaries = features(X_test, summaries)

X_train_mean = features_mean(X_train)
X_test_mean = features_mean(X_test)

#### Running the model

A little bit of fun has arrived. Here I create a function in order to run the decision forest with different number of nodes. As you know, the number of nodes in a forest is extremely important as it allows to control for overfitting. Let's have a look at all the arguments:

- **n_estimators** : this is a list and as I anticipate, with this we can control the number of nodes
- **X_traind and X_test** : because we used 2 different kinds of features engineering, here we can choose which one to use
- **y_train and y_test** : regardless wich kind of training data we use, we will always use the same test set and for this reason these arguments have already default values
- **random_state** : simply a way of controlling for randomness when looking for the best split at each node. This is usefull in order to be able to reproduce the accuracy.

I leave you [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) the official documentation about decision forest.

In conclusion, not only this function allows us to choose the training data and the number of nodes, but it saves for each number of nodes its accuracy

In [10]:
## this function take as argument a training and a validation data set and return the accuracy based on 
# the number of nodes, which is encode here as n_estimators

def run_forest(n_estimators ,X_train, X_test, y_train = y_train, y_test = y_test , random_state = 333):
    
    acc=[] # list of accuracy which depends on the hyperparameter n_estimators

    for num_features in n_estimators: # hyperparameters to change with higher number
        forest = RandomForestClassifier(n_estimators=num_features, 
                               bootstrap = True,
                               max_features = 'sqrt',
                                random_state = random_state)
        fitted_model=forest.fit(X_train,y_train)
        prediction=fitted_model.predict(X_test)
        accuracy=accuracy_score(y_test,prediction)
        solution = (num_features,accuracy)
        acc.append(solution)
        
    return acc

Let's run the model, here we can define the lists to put as **n_estimators** argument. My machine cannot handle all these data, for the sake of the example, I just put 1 node for each type of features engineering. You can try with [50,100,200]  and you will see that the accuracy is going to increase, although it cannot go over 20%. Not too much eh? For this we have DL ;).

In [11]:
#set up different number of nodes 
n_estimators_mean = [1]
n_estimators_summaries = [1]

In [12]:
# accuracy for the mean
accuracy_mean = run_forest(n_estimators_mean,X_train_mean,X_test_mean)
print(accuracy_mean)

[(1, 0.05715792248879515)]


In [13]:
# accuracy for the summaries
accuracy_summaries = run_forest(n_estimators_summaries,X_train_summaries,X_test_summaries)
print(accuracy_summaries)


[(1, 0.07835486422356973)]


### 2D Convolutional Neural Network 

Let's clean everything by uploading in new variables our data

In [15]:
# change name of variables 
train_X = training_data
test_X = test_data
train_Y = labels

Now we are going to use the Conv2D function, which is mainly use for image classification, but also in this case we can reach an important accuracy (more than 90%). Because of that, we need to reshape our data by adding a fictitious size

In [16]:
# reshape in four dimensions for input CNN
train_X = train_X.reshape(-1, 99,13, 1)
test_X = test_X.reshape(-1, 99,13, 1)
train_X.shape, test_X.shape

((94824, 99, 13, 1), (11005, 99, 13, 1))

In [17]:
# transform data type in float
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

With CNN we are going to predict a probability for each possible class for each example and for this we need to change again our target. We will use the [to_categorical](https://keras.io/utils/) function. This function creates for each target a list with as many position as number of classes-1 (because it starts from 0) and each *ith* position represents the *ith* class

In [18]:
# transform the labels
train_Y_one_hot = to_categorical(train_Y)

For example, here we can see that the first target is the 22° class, this is because in the 21th position we have 1, while in the others we have 0

In [ ]:
train_Y_one_hot[0]

In [19]:
## CREATE THE VALIDATION SET 
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, 
                                                           random_state=13)

In [20]:
# check all the shape
train_X.shape,valid_X.shape,train_label.shape,valid_label.shape

((75859, 99, 13, 1), (18965, 99, 13, 1), (75859, 35), (18965, 35))

#### First attempt

As you might know, before running CNN we need to set up some hyperparameters

In [21]:
            #----------------------------------- first attempt ---------------------------------#

# set up hyperparameters 
batch_size = 64
epochs = 1
num_classes = 35 # fix
np.random.seed(222)

Here our model: 3 CNN2D layers and a fully connected layer before the output layer

In [22]:
# set up the layers 
fashion_model = Sequential()

fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(99,13,1),padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))

fashion_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

fashion_model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))  
fashion_model.add(Dense(num_classes, activation='softmax'))

In [23]:
fashion_model.compile(loss=keras.losses.categorical_crossentropy, 
                      optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [24]:
## check the summary
fashion_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 99, 13, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 99, 13, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 7, 32)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 7, 64)         18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 50, 7, 64)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 4, 64)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 4, 128)        73856     
__________

Let's run. Also here, I just use one epoch for the sake of the example. You should try with at least 20 epochs

In [25]:
## train and test the accuracy in the validation set
fashion_train = fashion_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,
                                  verbose=1,validation_data=(valid_X, valid_label))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 75859 samples, validate on 18965 samples
Epoch 1/1





75859/75859 [==============================] - 260s 3ms/step - loss: 1.2988 - acc: 0.6328 - val_loss: 0.6284 - val_acc: 0.8168


In conclusion, with this model you should reach 85% accuracy in validation. Not bad eh? However, we can go much more further.

#### Second attempt

Here we try to control overfitting by introducing the function and we hope to improve accuracy. Spoiler alert: we did!

To put it simply, during training, some number of layer outputs are randomly ignored or "dropped out." For this we use less parameters and as a result we can control overfitting

In [27]:
# set up hyperparameters 
batch_size = 124
epochs = 1
num_classes = 35 # fix
np.random.seed(222)

In [28]:
## set up the dropout to improve accuracy
fashion_model = Sequential()
fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(99,13,1),padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.4))

fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))  
fashion_model.add(Dropout(0.3))
fashion_model.add(Dense(num_classes, activation='softmax'))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [29]:
fashion_model.compile(loss=keras.losses.categorical_crossentropy, 
                      optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])

In [30]:
fashion_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 99, 13, 32)        320       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 99, 13, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 50, 7, 32)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 7, 32)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 50, 7, 64)         18496     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 50, 7, 64)         0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 25, 4, 64)         0         
__________

In [31]:
fashion_train = fashion_model.fit(train_X, train_label, 
                                  batch_size=batch_size,epochs=epochs,verbose=1,
                                  validation_data=(valid_X, valid_label))

Train on 75859 samples, validate on 18965 samples
Epoch 1/1
75859/75859 [==============================] - 366s 5ms/step - loss: 2.4425 - acc: 0.3238 - val_loss: 1.2589 - val_acc: 0.6495


In conclusion, with this model you should reach at least 90% accuracy.

### 1D Convolutional Neural Network

In [33]:
#go back to two three dimensions
train_X = training_data
test_X = test_data
train_Y = labels

train_X = train_X.reshape(-1, 99,13)
test_X = test_X.reshape(-1, 99,13)

# transform the labels
train_Y_one_hot = to_categorical(train_Y)

train_X.shape, test_X.shape

((94824, 99, 13), (11005, 99, 13))

In [34]:
## CREATE THE VALIDATION SET 
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, 
                                                           random_state=13)

In [35]:
# set up hyperparameters 
batch_size = 256
epochs = 1
num_classes = 35 # fix
np.random.seed(222)

In [36]:
fashion_model = Sequential()

fashion_model.add(Conv1D(64, kernel_size=6,activation='relu',padding='same',input_shape=(99,13)))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))

fashion_model.add(Dropout(0.2))
fashion_model.add(Conv1D(128, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))

fashion_model.add(Conv1D(128, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(128, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))


fashion_model.add(Conv1D(128, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(256, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(256, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(256, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(512, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(1024, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv1D(1024, kernel_size=6, activation='relu',padding='same'))
fashion_model.add(BatchNormalization())
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling1D(pool_size=2,padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Flatten())
fashion_model.add(Dense(1024, activation='relu'))
fashion_model.add(LeakyReLU(alpha=0.1))           
fashion_model.add(Dropout(0.2))
fashion_model.add(Dense(num_classes, activation='softmax'))

In [37]:
fashion_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 99, 64)            5056      
_________________________________________________________________
batch_normalization_1 (Batch (None, 99, 64)            256       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 99, 64)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 64)            0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 50, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 128)           49280     
_________________________________________________________________
batch_normalization_2 (Batch (None, 50, 128)           512       
__________

In [38]:
fashion_model.compile(loss=keras.losses.categorical_crossentropy, 
                      optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])

In [39]:
fashion_train = fashion_model.fit(train_X, train_label, 
                                  batch_size=batch_size,epochs=epochs,verbose=1,
                                  validation_data=(valid_X, valid_label))

Train on 75859 samples, validate on 18965 samples
Epoch 1/1
75859/75859 [==============================] - 1961s 26ms/step - loss: 2.5409 - acc: 0.2798 - val_loss: 1.8575 - val_acc: 0.4542


In [40]:
                # ----------------------- PREDICTION ON TEST SET ---------------------- #

In [48]:
#### make prediction on the test set
test_prediction = fashion_model.predict(test_X) # take the prediction from the model
test_prediction = np.argmax(np.round(test_prediction),axis=1) # take the index of the maximum probability
test_prediction = encoder.inverse_transform(test_prediction) # from number label to word
print(test_prediction)

['backward' 'yes' 'seven' ... 'five' 'backward' 'backward']


In [49]:
## add the column of prediction to the test csv
test_csv = pd.read_csv("test.csv", delimiter = ",",index_col=None)
test_csv['word'] = test_prediction

In [51]:
## save the csv
out_csv = "result.csv"
test_csv.to_csv(out_csv,index=None,header=True)
#export_csv = test_csv.to_csv ("~/Desktop/Tilburg_university/Block2/export_dataframe.csv", 
#                              index = None, header=True)